In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

model_name = "baichuan-inc/Baichuan2-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_name)

prompts = [
    "中国的首都是哪里？",
    "中国的首都是哪裡？",
    "中国 的首都是哪里？",
    "中国的首都是那里？",
    "中国的首都是哪？",
    "中国的capital是哪？",
    "中国的首府是哪里？",
    "Where is the capital of China?",
]

for prompt in prompts:
    messages = []
    messages.append({"role": "user", "content": prompt})
    response = model.chat(tokenizer, messages)
    print(f"输入：{prompt}\n输出：{response.strip()}\n")

输入：中国的首都是哪里？
输出：中国的首都是北京。

输入：中国的首都是哪裡？
输出：中国的首都是北京。

输入：中国 的首都是哪里？
输出：中国的首都是北京。

输入：中国的首都是那里？
输出：中国的首都是北京。

输入：中国的首都是哪？
输出：中国的首都是北京。

输入：中国的capital是哪？
输出：中国的首都，也就是政治中心是北京。而经济中心则是上海。

输入：中国的首府是哪里？
输出：中国的首都为北京。

输入：Where is the capital of China?
输出：The capital of China is Beijing.

